In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import argparse
import sys
import numpy as np
from skimage.transform import resize

from tensorflow.examples.tutorials.mnist import input_data

import tensorflow as tf

FLAGS = None

In [8]:
rng = np.random.RandomState()
crop = 8
ds_factor = 2.

def new_shape(crop, ds_factor):
    return int(np.around((28 - crop) / ds_factor))

def transform(X, crop=2, ds_factor=2.):
    Xp = X.reshape(X.shape[0], 28, 28)
    Xp = Xp[:, crop:28-crop, crop:28-crop]
    new = new_shape(crop, ds_factor)
    Xp = resize(Xp, (Xp.shape[0], new, new))
    return Xp.reshape(Xp.shape[0], -1)

In [9]:
mnist = input_data.read_data_sets('/tmp/tensorflow/mnist/input_data/', one_hot=True)
new_dim = new_shape(crop, ds_factor) **2
# Create the model
x = tf.placeholder(tf.float32, [None, new_dim])
p = tf.Variable(tf.zeros((new_dim * 10) + 10))
W = tf.reshape(p[:new_dim * 10], [new_dim, 10])
b = tf.reshape(p[new_dim * 10:], [10])

y = tf.matmul(x, W) + b

# Define loss and optimizer
y_ = tf.placeholder(tf.float32, [None, 10])

cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y))

Extracting /tmp/tensorflow/mnist/input_data/train-images-idx3-ubyte.gz
Extracting /tmp/tensorflow/mnist/input_data/train-labels-idx1-ubyte.gz
Extracting /tmp/tensorflow/mnist/input_data/t10k-images-idx3-ubyte.gz
Extracting /tmp/tensorflow/mnist/input_data/t10k-labels-idx1-ubyte.gz


In [10]:
opt = tf.train.GradientDescentOptimizer(learning_rate=1.)
grad, var = opt.compute_gradients(cross_entropy, var_list=[p])[0]
grad2 = tf.reshape(grad, (grad.shape[0], 1))

# Meaningless outside pixel make Hessian not invertable, cropping would fix need for tf.eye
hess = tf.squeeze(tf.hessians(cross_entropy, p)) #+ .0001 * tf.eye((new_dim * 10) + 10)
grad_prime = tf.squeeze(tf.matmul(tf.matrix_inverse(hess), grad2))
train_step = opt.apply_gradients([(grad_prime, var)])

In [11]:
sess = tf.InteractiveSession()
tf.global_variables_initializer().run()

In [12]:
correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

# Train
print('training')
for ii in range(50000):
    batch_xs, batch_ys = mnist.train.next_batch(1000)
    sess.run(train_step, feed_dict={x: transform(batch_xs, crop, ds_factor), y_: batch_ys})
    if ii % 100 == 0:
        print(sess.run(accuracy, feed_dict={x: transform(mnist.test.images, crop, ds_factor),
                                  y_: mnist.test.labels}))
        print(sess.run(accuracy, feed_dict={x: transform(batch_xs, crop, ds_factor), y_: batch_ys}))
        print()

print('test')
print(sess.run(accuracy, feed_dict={x: transform(mnist.test.images, crop, ds_factor),
                                  y_: mnist.test.labels}))

training


/home/jesse/anaconda3/envs/tf35/lib/python3.5/site-packages/skimage/transform/_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "


0.4647
0.533



InvalidArgumentError: Input is not invertible.
	 [[Node: MatrixInverse_1 = MatrixInverse[T=DT_FLOAT, adjoint=false, _device="/job:localhost/replica:0/task:0/device:GPU:0"](Squeeze_2)]]

Caused by op 'MatrixInverse_1', defined at:
  File "/home/jesse/anaconda3/envs/tf35/lib/python3.5/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/jesse/anaconda3/envs/tf35/lib/python3.5/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/jesse/anaconda3/envs/tf35/lib/python3.5/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/home/jesse/anaconda3/envs/tf35/lib/python3.5/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/home/jesse/anaconda3/envs/tf35/lib/python3.5/site-packages/ipykernel/kernelapp.py", line 477, in start
    ioloop.IOLoop.instance().start()
  File "/home/jesse/anaconda3/envs/tf35/lib/python3.5/site-packages/zmq/eventloop/ioloop.py"